In [4]:
from paho.mqtt import client as mqtt_client
import time
from dotenv import dotenv_values

In [5]:
storage=[]
imei_dict={}
device_count=-1
def convert_str_to_dict(data):
    x=data.split(",")
    l=[]
    field,value=x[0][1:5:],int(x[0][6:-1:])
    field=field[1:-1]
    l.append(value)
    for i in range(1,len(x)-1):
        field,value=x[i].split(":")
        field=field[1:-1]
        if(value[0]!='"'):
            if(i==6):
                value=int(value)
            else:
                value=float(value)
        else:
            value=value[1:-1]
        l.append(value)
    field,value=x[-1][0:4:],int(x[-1][5:6:])
    field=field[1:-1]
    value=int(value)
    l.append(value)
    return l
def soc_calculator(EBV_value):
    if(EBV_value>52.95):
        return  90
    elif(EBV_value>52.4):
        return 80
    elif(EBV_value>51.85):
        return 70
    elif(EBV_value>51.3):
        return 60
    elif(EBV_value>50.75):
        return 50
    elif(EBV_value>50.2):
        return 40
    elif(EBV_value>49.65):
        return 30
    elif(EBV_value>49):
        return 20
    elif(EBV_value>48.55):
        return 10
    elif(EBV_value>48):
        return 0
    else:
        return -1
    


In [6]:

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    #print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("emb/f2s/track/#",0)
# The callback for when a PUBLISH message is received from the server.
def to_storage(x,device_count,client):
    if(x[6]==0):
        soc=soc_calculator(x[5])
        x[10]=soc
        if(x[1] not in imei_dict.keys()):
            imei_dict[x[1]] = len(storage)
            storage.append([])
            storage[imei_dict[x[1]]].append(x)
        else:
            ping_count=len(storage[imei_dict[x[1]]])
            if(ping_count<2):
                storage[imei_dict[x[1]]].append(x)
            else:
                storage[imei_dict[x[1]]][0]=storage[imei_dict[x[1]]][1]
                storage[imei_dict[x[1]]][1]=x
        on_publish(client,x[1],x[10])
def on_message(client, userdata, message):
    #print("#")
    #print(message.topic+" "+str(message.payload))
    x=convert_str_to_dict(str(message.payload.decode('utf-8')))
    #print("#",x)
    to_storage(x,device_count,client)
    #print(storage)
def on_publish(client,imei,soc):
        topic="emb/soc/"+imei
        print(topic)
        info=client.publish(topic=topic,payload=soc,qos=0)

secrets= dotenv_values(".env")


broker_address = secrets["broker_address"]  
port = int(secrets["port"])                         
user = secrets["user_name"]                 
password = secrets["Password"]           

client = mqtt_client.Client("trial")
client.username_pw_set(user, password=password)
client.on_connect = on_connect
client.on_message = on_message



client.connect(broker_address, port)
client.loop_start()
#client.subscribe("emb/f2s/track/#",0)
# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.

time.sleep(300)
client.loop_stop()

emb/soc/869523059872433


In [ ]:

print(imei_dict)
print(storage)


{'863005061009465': 0, '869523059853011': 1, '869523059854613': 2, '869523059854514': 3, '863005061001322': 4, '869523059873415': 5, '863005061009499': 6, '863005061009598': 7, '863005060984627': 8}
[[[168864920, '863005061009465', 31.0, 77.65028, 12.879769, 53.12, 0, 1561.71, 11.8, '000000000000', 90, 0.0, 4], [168864932, '863005061009465', 30.0, 77.650283, 12.879765, 53.12, 0, 1561.71, 11.8, '000000000000', 90, 0.0, 4]], [[168864921, '869523059853011', 9.0, 77.649214, 12.916753, 0.7, 0, 8.65, 10.9, '000000000000', -1, 0.0, 4], [168864933, '869523059853011', 9.0, 77.649214, 12.916753, 0.7, 0, 8.65, 10.9, '000000000000', -1, 0.0, 4]], [[168864922, '869523059854613', 23.0, 77.650262, 12.908942, 0.72, 0, 0.11, 11.9, '000101160301', -1, 100.0, 4], [168864934, '869523059854613', 20.0, 77.650262, 12.908942, 0.72, 0, 0.11, 11.9, '000101160301', -1, 100.0, 4]], [[168864910, '869523059854514', 19.0, 0.0, 0.0, 0.7, 0, 0.19, 11.6, '000000000000', -1, 0.0, 4]], [[168864912, '863005061001322', 19.